In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import ascii
from astropy.table import Column, hstack, Table, TableMergeError, vstack
import astropy.units as u
import os

%run Utility.ipynb

# The parameters provided to this function are as is:
# CAT_1 -- An astropy.table object containing your first catalog
# CAT_1_RA -- The name of the column in your first catalog with right-ascenscion values
# CAT_1_DEC -- The name of the column in your first catalog with declination values
# CAT_2 -- An astropy.table object containing your second catalog
# CAT_2_RA -- The name of the column in your second catalog with right-ascenscion values
# CAT_2_DEC -- The name of the column in your second catalog with declination values
# K -- The cross-matching radius (in arc-seconds)
# NTHNEIGHBOR -- Which closest cross-match to the source to save (sometimes, a catalog is cross-matched with itself in order to strip down repeated sources - in this case, it is useful to change NTHNEIGHBOR = 2)
# The function returns a new astropy.table object with only the cross-matched sources between the first and second catalogs, along with all the columns from both of these catalogs
def cross_match(cat_1, cat_1_ra, cat_1_dec, cat_2, cat_2_ra, cat_2_dec, k = 2, nthneighbor = 1, sep = False):
    
    # Some of the catalogs have right-ascenscion written in hours instead of degrees
    # This will raise a TypeError, which this try-and-except routine takes care of
    try:
        
        # Transforms the right ascenscion and declination columns of the first catalog into SkyCoord arrays
        c_1 = SkyCoord(ra = cat_1[cat_1_ra].astype(float) * u.degree, dec = cat_1[cat_1_dec].astype(float) * u.degree)
    
    except TypeError:
        
        ra = cat_1[cat_1_ra]
        
        dec = cat_1[cat_1_dec]
        
        # Transforms the right ascenscion and declination columns of the second catalog into SkyCoord arrays
        c_1 = SkyCoord(ra = ra, dec = dec, unit = (u.hourangle, u.deg), frame = 'icrs')
    
    try:
        
        # Transforms the right ascenscion and declination columns of the second catalog into SkyCoord arrays
        c_2 = SkyCoord(ra = cat_2[cat_2_ra].astype(float) * u.degree, dec = cat_2[cat_2_dec].astype(float) * u.degree)
    
    except TypeError:
        
        ra = cat_2[cat_2_ra]
        
        dec = cat_2[cat_2_dec]
        
        # Transforms the right ascenscion and declination columns of the second catalog into SkyCoord arrays
        c_2 = SkyCoord(ra = ra, dec = dec, unit = (u.hourangle, u.deg), frame = 'icrs')
    
    # The index-array of the rows in the secondary catalog which matched with each row of the main catalog
    # The 2-D distance between the source in the main catalog with the matched source in the secondary catalog
    # The 3-D distance between the source in the main catalog with the matched source in the secondary catalog
    idx, d2d, d3d = c_1.match_to_catalog_sky(c_2, nthneighbor = nthneighbor)
    
    # Determines the max range at which a pair is considered to be properly matched (in arc-seconds)
    max_sep = k * u.arcsec
    
    # The index-array of the matches with a distance between the two objects inferior to the maximum allowed separation set by the MAX_SEP variable
    idx_sep = d2d < max_sep
    
    # The rows of the first catalog with a match
    cat_1_m = cat_1[idx_sep]
    
    # The rows of the second catalog with a match
    cat_2_m = cat_2[idx[idx_sep]]
    
    # Throughout multiple cross-matches, some column names may repeat themselves in the different catalogs
    # The HSTACK function appends a sufix in column names common to both catalogs being cross-matched using the TABLE_NAMES variable
    # As it is now, a column named EXAMPLE which is common to both catalogs will be renamed to EXAMPLE_ and EXAMPLE_2, each corresponding to the first and second catalog, respectively
    # For the case where you are cross-matching a catalog which already is the product of this CROSSMATCH function, it may happen that there are already EXAMPLE_ and EXAMPLE_2 columns
    # This try-and-except routine accounts for that and the new column will be named EXAMPLE_3 (change from 3 to 4, 5, 6, etc., as is needed)
    try:
        
        # Stackes the matched rows horizontally, preserving all the information pertaining to both catalogs about the rows
        tabular_f = hstack([cat_1_m, cat_2_m], table_names = ["", "2"])
    
    except TableMergeError:
        
        i = 3
        
        # Stackes the matched rows horizontally, preserving all the information pertaining to both catalogs about the rows
        #tabular_f = hstack([cat_1_m, cat_2_m], table_names = ["", "3"])
        tabular_f = iter_col_name(cat_1_m, cat_2_m, i)
    
    if sep == False:
        
        return tabular_f
    
    else:
        
        d2d_sep = d2d[d2d < max_sep].degree
        
        tabular_f['d2d'] = d2d_sep
        
        return tabular_f

# Use this function to stack all catalogs in a target folder into a single one
def stack_catalogs(path, save_path):
    
    full_cat = Table()
    
    #Encodes the path of the dataset to be used by other OS methods
    folder = os.fsencode(path)
    
    for item in os.listdir(folder):
        
        item = os.fsdecode(item) #Decodes the current item so it becomes a string
        
        item = path + item #To comply with the func() function parameter
        
        print(item)
        
        temp_cat = ascii.read(item)
        
        full_cat = vstack([full_cat, temp_cat])
    
    ascii.write(full_cat, save_path, format = 'csv', overwrite = True)

# For when attributing repeating column names when cross-matching (It's used by the cross-matching function)
def iter_col_name(cat_1, cat_2, i):
    
    i = str(i)
    
    try:
        
        tabular_f = hstack([cat_1, cat_2], table_names = ["", i])
    
    except TableMergeError:
        
        i = int(i) + 1
        
        tabular_f = iter_col_name(cat_1, cat_2, i)
    
    return tabular_f